In [1]:
import numpy as np
import matplotlib.pyplot as plt
import CTS as cts
from scipy.constants import c

plt.rcParams.update({'font.size': 16})
%matplotlib qt

In [2]:
freq_in = 500e9 # Hz
lambda_in = c / freq_in
ne = [1e13, 1e12]
Te = [3,10]
Ti = [1,4]

In [3]:
fig, ax = plt.subplots()
for i in range(len(ne)):
    alpha, omega_arr, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=500, ions="H+")
    ax.plot(omega_arr, Skw, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
ax.legend()

for i in range(len(ne)):
    alpha, omega_arr, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=500, ions="He+")
    ax.plot(omega_arr, Skw, '--',label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
ax.legend()